In [20]:
# 5.1 MNIST数据处理
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../datasets/MNIST_data/", one_hot=True)

Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz


Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
print("Training data size: ", mnist.train.num_examples)
print("Validating data size: ", mnist.validation.num_examples)
print("Testing data size: ", mnist.test.num_examples)

Training data size:  55000
Validating data size:  5000
Testing data size:  10000


In [22]:
print("Example training data: ", mnist.train.images[0] )
print("Example training data label: ", mnist.train.labels[0])

Example training data:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

In [23]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)    # 从train的集合中选取batch_size个训练数据。
print("X shape:", xs.shape)                       
print("Y shape:", ys.shape)

X shape: (100, 784)
Y shape: (100, 10)


In [5]:
# 5.2 神经网络模型训练及不同模型结果对比
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST 数据集相关的常数 
INPUT_NODE = 784                        # 输入的节点数，等于图片像素
OUTPUT_NODE = 10                        # 输出的节点数，等于类别的数目
                                        # 区分0~9的10个数字，输出层节点数是10

# 配置神经网络的参数
LAYER1_NODE = 500                       # 设置隐藏层的节点数500
BATCH_SIZE = 100                        # 一个训练batch中的训练数据个数，随机梯度下降
LEARNING_RATE_BASE = 0.8                # 基础的学习率
LEARNING_RATE_DECAY = 0.99              # 学习率的衰减率
REGULARIZATION_RATE = 0.0001            # 正则化系数
TRAINING_STEPS = 5000                  # 训练轮数
MOVING_AVERAGE_DECAY = 0.99             # 滑动平均衰减率
MODEL_SAVE_PATH = "MNIST_model/"
MODEL_NAME = "mnist_model"

C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class ==None:
        # 计算隐藏层的前向传播结果，使用ReLU激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        # 计算输出层的前夕传播结果，计算损失函数时一并计算softmax函数
        # 所以不需要加入激活函数
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        # 首先使用avg_class.average计算变量的滑动平均值
        # 计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [26]:
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name = 'y-input')
    
    # 生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev = 0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    # 生成输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev = 0.1)) 
    biases2 = tf.Variable(tf.constant(0.1, shape = [OUTPUT_NODE]))
    
    # 计算在当前参数下神经网络前向传播的结果，不用参数滑动平均值
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义村相互训练轮数的变量
    global_step = tf.Variable(0, trainable = False)
    
    # 初始化滑动平均类，加快训练早起变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 取值维护滑动平均值
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵作为损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularization
    
    # 设置指数衰减率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY)
    
    # 使用tf.train.GradientDescentOptimizer优化损失函数 包含交叉熵和L2正则损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # 每训练一次更新一次参数
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name = 'train')
        
    # 验证前向传播的结果
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话
    with tf.Session() as sess:
        
        tf.global_variables_initializer().run()
        # 准备验证数据。
        validate_feed = {x:mnist.validation.images,
                         y_: mnist.validation.labels}
        # 准备测试数据
        test_feed = {x:mnist.test.images, y_: mnist.test.labels}
        
        # 迭代训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training steps, validation accuracy using average model is %g " % (i, validate_acc))
            # 产生这一轮batch的训练数据
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict = {x: xs, y_: ys})
        
        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

In [27]:
def main(argv = None):
    # mnist = input_data.read_data_sets("../../datasets/MNIST_data/",one_hot=True)
    train(mnist)
    
if __name__ == '__main__':
    tf.app.run()

After 0 training steps, validation accuracy using average model is 0.0974 


After 1000 training steps, validation accuracy using average model is 0.9762 


After 2000 training steps, validation accuracy using average model is 0.9818 


After 3000 training steps, validation accuracy using average model is 0.9816 


After 4000 training steps, validation accuracy using average model is 0.983 


After 5000 training steps, test accuracy using average model is 0.9819 
After 4999 training steps, validation accuracy using average model is 0.9834, test accuracy using average model is 0.9819


SystemExit: 

C:\Users\adward\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [37]:
# 不使用正则化
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    loss = cross_entropy_mean
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

def main(argv=None):
    #mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.0788 


After 1000 training step(s), validation accuracy using average model is 0.9774 


After 2000 training step(s), validation accuracy using average model is 0.981 


After 3000 training step(s), validation accuracy using average model is 0.9836 


After 4000 training step(s), validation accuracy using average model is 0.985 


After 5000 training steps, test accuracy using average model is 0.9825 
After 4999 training steps, validation accuracy using average model is 0.9848, test accuracy using average model is 0.9825


In [30]:
# 不使用指数衰减的学习率
LEARNING_RATE = 0.1  

def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

def main(argv=None):
    # mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.099 


After 1000 training step(s), validation accuracy using average model is 0.9492 


After 2000 training step(s), validation accuracy using average model is 0.9654 


After 3000 training step(s), validation accuracy using average model is 0.9706 


After 4000 training step(s), validation accuracy using average model is 0.9762 


After 5000 training steps, test accuracy using average model is 0.9739 
After 4999 training steps, validation accuracy using average model is 0.9762, test accuracy using average model is 0.9739


In [32]:
# 不使用激活函数
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

def main(argv=None):
    # mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.1258 


After 1000 training step(s), validation accuracy using average model is 0.9774 


After 2000 training step(s), validation accuracy using average model is 0.9838 


After 3000 training step(s), validation accuracy using average model is 0.9826 


After 4000 training step(s), validation accuracy using average model is 0.983 


After 5000 training steps, test accuracy using average model is 0.9832 
After 4999 training steps, validation accuracy using average model is 0.9832, test accuracy using average model is 0.9832


In [35]:
# 不使用隐藏层
def inference_nohidenlayer(input_tensor, avg_class, weights1, biases1):
    # 使用滑动平均类
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return layer1

    else:
        # 不使用滑动平均类
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return layer1

def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 生成输出层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, OUTPUT_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference_nohidenlayer(x, None, weights1, biases1)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference_nohidenlayer(x, variable_averages, weights1, biases1)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularaztion = regularizer(weights1)
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

def main(argv=None):
    #mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.0892 


After 1000 training step(s), validation accuracy using average model is 0.6552 


After 2000 training step(s), validation accuracy using average model is 0.6572 


After 3000 training step(s), validation accuracy using average model is 0.66 


After 4000 training step(s), validation accuracy using average model is 0.6584 


After 5000 training steps, test accuracy using average model is 0.6656 
After 4999 training steps, validation accuracy using average model is 0.6578, test accuracy using average model is 0.6656


In [36]:
# 不使用滑动平均类
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数及相关的滑动平均类 
    global_step = tf.Variable(0, trainable=False)
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数
    with tf.control_dependencies([train_step]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
        # 训练结束后，检验最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("After %d training steps, test accuracy using average model is %g " % (TRAINING_STEPS, test_acc))
        # 5.2.2 使用验证数据集判断模型效果
        # 计算滑动平均模型在测试数据和验证数据上的正确率
        validate_acc = sess.run(accuracy, feed_dict = validate_feed)
        # test_acc = sess.run(accuracy, feed_dict = test_feed)
        
        #输出正确率信息
        print("After %d training steps, validation accuracy using average model is %g, test accuracy using average model is %g" % (i, validate_acc, test_acc) )

def main(argv=None):
    #mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.1228 


After 1000 training step(s), validation accuracy using average model is 0.9668 


After 2000 training step(s), validation accuracy using average model is 0.9774 


After 3000 training step(s), validation accuracy using average model is 0.9796 


After 4000 training step(s), validation accuracy using average model is 0.9798 


After 5000 training steps, test accuracy using average model is 0.9818 
After 4999 training steps, validation accuracy using average model is 0.98, test accuracy using average model is 0.9818


In [25]:
# 5.3 变量管理
# tf.get_variable() 与 tf.Variable()作用相同
import tensorflow as tf

p = tf.get_variable("p", shape=[1], initializer=tf.constant_initializer(1.0))

q = tf.Variable(tf.constant(1.0, shape =[1]), name="q")

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(p))
    print(sess.run(q))

[1.]
[1.]


In [30]:
# 在名字为"foo"的命名空间内创建名字为v的变量并初始化
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1], initializer = tf.constant_initializer(1.0))
    
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v==v1)

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\adward\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1269, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\adward\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2506, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\adward\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [34]:
# tf.variabel_scope()嵌套  
with tf.variable_scope("root"):
    # tf.get_variable_scope().resue获取上下文管理器中参数取值
    print(tf.get_variable_scope().reuse)
    
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        
        with tf.variable_scope("bar"):
            print(tf.get_variable_scope().reuse)
            
    print(tf.get_variable_scope().reuse)

False
True
True
False


In [2]:
v1 = tf.get_variable("v", [1])
print(v1.name)                                   #v:0

with tf.variable_scope("foo"):
    v2 = tf.get_variable("v",[1])
    print(v2.name)                               #foo/v:0 
    
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v", [1])
        print(v3.name)                           #foo/bar/v:0 
    
    v4 = tf.get_variable("v1",[1])
    print(v4.name)                               #foo/v1:0 
    
with tf.variable_scope("",reuse=True):
    v5 = tf.get_variable("foo/bar/v", [1])
    
    print(v5==v3)                                #True
    v6 = tf.get_variable("foo/v1", [1])
    print(v6==v4)                                #True

v:0
foo/v:0
foo/bar/v:0
foo/v1:0
True
True


In [7]:
## 改进前向传播算法
def inference (input_tensor, reuse=False):
    # 定义第一次神经网络的变量和前向传播过程
    with tf.variable_scope('layer1', reuse=reuse):
        # 根据船机那里的reuse判断是否是已经穿件好的变量
        weights = tf.get_variable("weights", [INPUT_NODE, LAYER1_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases",[LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
        
    # 类似定义第二层和传播过程
    with tf.variable_scope('layer2', reuse=reuse):
        weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases",[OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    return layer2

x = tf.placeholder(tf.float32, [None, INPUT_NODE], name = 'x-input')
y = inference(x)

In [12]:
# 5.4 TensorFlow模型持久化
import tensorflow as tf
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
# 声明tf.train.Saver类用于保护模型
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    # 将模型保存到/path/to/model/model.ckpt文件
    saver.save(sess, "./model.ckpt")

In [15]:
# 保存模型并加载
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

saver = tf.train.Saver()

with tf.Session() as sess:
    # 将模型保存到/path/to/model/model.ckpt文件
    saver.restore(sess, "./model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from ./model.ckpt


NotFoundError: Key v2_12 not found in checkpoint
	 [[Node: save_13/RestoreV2_18 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_13/Const_0_0, save_13/RestoreV2_18/tensor_names, save_13/RestoreV2_18/shape_and_slices)]]

Caused by op 'save_13/RestoreV2_18', defined at:
  File "C:\Users\adward\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\adward\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\adward\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\adward\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\adward\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\adward\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in s

In [16]:
import tensorflow as tf
# 直接加载持久化的图
saver = tf.train.import_meta_graph("./model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "./model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from ./model.ckpt


[3.]


In [17]:
# 使用字典重命名变量
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
# 将other-v1加载到v1，other-v2加载到v2
saver = tf.train.Saver({"v1": v1, "v2": v2})

In [31]:
# 使用saver保存滑动平均模型
v = tf.Variable(0, name = "v")
# 在没有声明滑动平均模型只有一个变量v时，只会输出“v:0”
for variables in tf.global_variables():
    print(variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
# apply添加影子变量
maintain_average_op = ema.apply(tf.global_variables())

for variables in tf.global_variables(): 
    print(variables.name)
    
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存的时候会将v:0  v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "../model/model2.ckpt")
    print(sess.run([v, ema.average(v)]))

v1:0
v2:0
v1_1:0
v2_1:0
v1_2:0
v2_2:0
v1_3:0
v2_3:0
v1_4:0
v2_4:0
v1_5:0
v2_5:0
v1_6:0
v2_6:0
v1_7:0
v2_7:0
v1_8:0
v2_8:0
v1_9:0
v2_9:0
v1_10:0
v2_10:0
v1_11:0
v2_11:0
v1_12:0
v2_12:0
v1_13:0
v2_13:0
v1:0
v2:0
v1_1:0
v2_1:0
v1_2:0
v2_2:0
v1_3:0
v2_3:0
v1_4:0
v2_4:0
v1_5:0
v2_5:0
v1_6:0
v2_6:0
v1_7:0
v2_7:0
v1_8:0
v2_8:0
v1_9:0
v2_9:0
v1_10:0
v2_10:0
other-v1:0
other-v2:0
v:0
v1/ExponentialMovingAverage:0
v2/ExponentialMovingAverage:0
v1_1/ExponentialMovingAverage:0
v2_1/ExponentialMovingAverage:0
v1_2/ExponentialMovingAverage:0
v2_2/ExponentialMovingAverage:0
v1_3/ExponentialMovingAverage:0
v2_3/ExponentialMovingAverage:0
v1_4/ExponentialMovingAverage:0
v2_4/ExponentialMovingAverage:0
v1_5/ExponentialMovingAverage:0
v2_5/ExponentialMovingAverage:0
v1_6/ExponentialMovingAverage:0
v2_6/ExponentialMovingAverage:0
v1_7/ExponentialMovingAverage:0
v2_7/ExponentialMovingAverage:0
v1_8/ExponentialMovingAverage:0
v2_8/ExponentialMovingAverage:0
v1_9/ExponentialMovingAverage:0
v2_9/ExponentialMo


v1_13/ExponentialMovingAverage_3:0
v2_13/ExponentialMovingAverage_3:0
v_5:0
v_6:0
v_7:0
v1/ExponentialMovingAverage_4:0
v2/ExponentialMovingAverage_4:0
v1_1/ExponentialMovingAverage_4:0
v2_1/ExponentialMovingAverage_4:0
v1_2/ExponentialMovingAverage_4:0
v2_2/ExponentialMovingAverage_4:0
v1_3/ExponentialMovingAverage_4:0
v2_3/ExponentialMovingAverage_4:0
v1_4/ExponentialMovingAverage_4:0
v2_4/ExponentialMovingAverage_4:0
v1_5/ExponentialMovingAverage_4:0
v2_5/ExponentialMovingAverage_4:0
v1_6/ExponentialMovingAverage_4:0
v2_6/ExponentialMovingAverage_4:0
v1_7/ExponentialMovingAverage_4:0
v2_7/ExponentialMovingAverage_4:0
v1_8/ExponentialMovingAverage_4:0
v2_8/ExponentialMovingAverage_4:0
v1_9/ExponentialMovingAverage_4:0
v2_9/ExponentialMovingAverage_4:0
v1_10/ExponentialMovingAverage_4:0
v2_10/ExponentialMovingAverage_4:0
v1_11/ExponentialMovingAverage_4:0
v2_11/ExponentialMovingAverage_4:0
v1_12/ExponentialMovingAverage_4:0
v2_12/ExponentialMovingAverage_4:0
v1_13/ExponentialMovingAv

AttributeError: 'Variable' object has no attribute '_initial_value'

In [32]:
v = tf.Variable(0, dtype=tf.float32, name="v")

# 通过变量重命名将原来变量v的滑动平均值直接赋值给v。
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "/model/model2.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from /model/model2.ckpt


In [34]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    with tf.gfile.GFile("model/combined_model.pb", "wb") as f:
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.


In [35]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "model/combined_model.pb"
   
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([3.], dtype=float32)]


In [1]:
# 持久层的存储
import tensorflow as tf
v1 = tf.Variable(tf.constant(1.0,shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0,shape=[1]), name="v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess,"../saved_model/model.ckpt")

# 加载持久层
with tf.Session() as sess:
    # sess.run(init_op)
    saver.restore(sess,"../saved_model/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from ../saved_model/model.ckpt


[3.]


In [14]:
# 加载持久化的图.meta
saver = tf.train.import_meta_graph("../saved_model/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "../saved_model/model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from ../saved_model/model.ckpt


[3.]


In [2]:
# 变量重命名
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})

In [3]:
# 滑动平均模型
v = tf.Variable(0, dtype=tf.float32, name="v")
for variables in tf.global_variables(): print(variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
for variables in tf.global_variables(): print(variables.name)

v1:0
v2:0
other-v1:0
other-v2:0
v:0
v1:0
v2:0
other-v1:0
other-v2:0
v:0
v1/ExponentialMovingAverage:0
v2/ExponentialMovingAverage:0
other-v1/ExponentialMovingAverage:0
other-v2/ExponentialMovingAverage:0
v/ExponentialMovingAverage:0


In [4]:
# 保存滑动平均模型
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存的时候会将v:0  v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "../saved_model/model2.ckpt")
    print(sess.run([v, ema.average(v)]))

[10.0, 0.099999905]


In [5]:
# 通过该持久层给变量v重命名直接赋予值
v = tf.Variable(0, dtype = tf.float32, name="v")
saver = tf.train.Saver({"v/ExponentialMovingAverage":v})
with tf.Session() as sess:
    saver.restore(sess, "../saved_model/model2.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from ../saved_model/model2.ckpt


0.099999905


In [4]:
import tensorflow as tf
# variables_to_restore
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)

print(ema.variables_to_restore())

saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "../saved_model/model.ckpt")
    print(sess.run(v))

{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>, 'v_2/ExponentialMovingAverage': <tf.Variable 'v_2:0' shape=() dtype=float32_ref>, 'v_1/ExponentialMovingAverage': <tf.Variable 'v_1:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from ../saved_model/model.ckpt


NotFoundError: Key v/ExponentialMovingAverage not found in checkpoint
	 [[Node: save_2/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_2/Const_0_0, save_2/RestoreV2/tensor_names, save_2/RestoreV2/shape_and_slices)]]

Caused by op 'save_2/RestoreV2', defined at:
  File "C:\Users\adward\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\adward\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\adward\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\adward\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\adward\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\adward\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, 

In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    with tf.gfile.GFile("../saved_model/combined_model.pb", "wb") as f:
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.


In [2]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "../saved_model/combined_model.pb"
   
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([3.], dtype=float32)]


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH="MNIST_model/"
MODEL_NAME="mnist_model"

def get_weight_variable(shape, regularizer): 
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if(regularizer != None): tf.add_to_collection('losses', regularizer(weights))
    return weights


def inference(input_tensor, regularizer, reuse):
    with tf.variable_scope('layer1', reuse = reuse):

        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope('layer2', reuse = reuse):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

def train(mnist):

    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer, False)
    global_step = tf.Variable(0, trainable=False)


    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    # cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_, 1), logits=y)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')


    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)


def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data/", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz


Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


After 1 training step(s), loss on training batch is 3.22999.


After 1001 training step(s), loss on training batch is 0.200413.


...

After 27001 training step(s), loss on training batch is 0.0402204.


After 28001 training step(s), loss on training batch is 0.0333728.


After 29001 training step(s), loss on training batch is 0.0338902.


SystemExit: 

C:\Users\adward\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 